In [48]:
import pandas as pd
training_data_filepath="training_data.csv"
training_data=pd.read_csv(training_data_filepath)
y=training_data.loan_default
data_features=['loan_id','age','education','proof_submitted','loan_amount','asset_cost','no_of_loans','no_of_curr_loans','last_delinq_none']
X=training_data[data_features]
X.head()

,loan_id,age,education,proof_submitted,loan_amount,asset_cost,no_of_loans,no_of_curr_loans,last_delinq_none
0,1033,28,1.0,Aadhar,777600.0,961200.0,0,0,0
1,6340,39,2.0,Aadhar,463848.0,1046532.0,5,3,0
2,3887,27,2.0,Aadhar,654984.0,865380.0,2,0,0
3,2654,29,2.0,VoterID,691716.0,867792.0,0,0,0
4,6915,37,1.0,Aadhar,796584.0,961200.0,0,0,0


In [49]:
from sklearn.model_selection import train_test_split
X_train,X_valid,y_train,y_valid=train_test_split(X,y,random_state=6)
categorical_cols=[cname for cname in X_train.columns if X_train[cname].dtype=='object']
numerical_cols=[cname for cname in X_train.columns if X_train[cname].dtype in ['int64','float64']]
all_cols=numerical_cols+categorical_cols
X_train_full=X_train[all_cols].copy()
X_valid_full=X_valid[all_cols].copy()

In [50]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,MaxAbsScaler
from sklearn.pipeline import Pipeline
numerical_transformer=Pipeline(steps=[('imputer',SimpleImputer(strategy='mean')),('scaler',MaxAbsScaler())])
categorical_transformer=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),('onehot',OneHotEncoder(handle_unknown='ignore'))
])
preprocessor=ColumnTransformer(
    transformers=[
        ('num',numerical_transformer,numerical_cols),
        ('cat',categorical_transformer,categorical_cols)
    ]
)

In [51]:
from xgboost import XGBClassifier
model=XGBClassifier(n_estimators=500,learning_rate=0.01,random_state=6)
from sklearn.metrics import accuracy_score
my_pipeline=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('model',model)
])
my_pipeline.fit(X_train_full,y_train)
preds=my_pipeline.predict(X_valid_full)
score=accuracy_score(preds,y_valid)
print(score)


C:\Users\khmam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


0.602846975088968


In [52]:
test_data_filepath="test_data.csv"
test_data=pd.read_csv(test_data_filepath)
preds2=my_pipeline.predict(test_data)
output = pd.DataFrame({'loan_id': test_data.loan_id,
                       'loan_default': preds2})
output.to_csv('submission.csv', index=False)